In [8]:
import pandas as pd
import psycopg2
df=pd.read_csv('/Users/peterscheinsohn/Downloads/sat-results.csv')
df.columns = df.columns.str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   dbn                              493 non-null    object 
 1   school name                      493 non-null    object 
 2   num of sat test takers           493 non-null    object 
 3   sat critical reading avg. score  493 non-null    object 
 4   sat math avg. score              493 non-null    object 
 5   sat writing avg. score           493 non-null    object 
 6   sat critical readng avg. score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB


In [16]:
import pandas as pd
import psycopg2
df=pd.read_csv('/Users/peterscheinsohn/Downloads/sat-results.csv')
df.columns = df.columns.str.lower()
df.head(50)

,dbn,school name,num of sat test takers,sat critical reading avg. score,sat math avg. score,sat writing avg. score,sat critical readng avg. score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0
5,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHOOL,112,332,557,316,332,414951,x345,NaN,3.0
6,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522,574,525,522,697107,NaN,78%,2.0
7,01M650,CASCADES HIGH SCHOOL,18,417,418,411,417,297600,NaN,92%,4.0
8,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624,604,628,624,881396,x234,NaN,NaN
9,02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,16,395,400,387,395,751293,NaN,78%,4.0


In [5]:
import pandas as pd
import difflib

# Load my original dataset
df = pd.read_csv('/Users/peterscheinsohn/Downloads/sat-results.csv')

# Convert all column names to lowercase
df.columns = df.columns.str.lower()

# Drop the 'contact_extension' column
df = df.drop(columns=['contact_extension'], errors='ignore')  # Use errors='ignore' to avoid error if column doesn't exist

# Identify and drop near-duplicate columns
def drop_near_duplicates(columns, threshold=0.9):
    to_drop = set()
    for i, col1 in enumerate(columns):
        for col2 in columns[i+1:]:
            similarity = difflib.SequenceMatcher(None, col1, col2).ratio()
            if similarity > threshold:
                to_drop.add(col2)
    return to_drop

# Get the set of columns to drop
columns_to_drop = drop_near_duplicates(df.columns)

# Drop the identified columns
df = df.drop(columns=columns_to_drop, errors='ignore')

# Convert the 'pct_students_tested' column from percentage string to float
if 'pct_students_tested' in df.columns:
    df['pct_students_tested'] = df['pct_students_tested'].str.rstrip('%').astype('float') / 100

# Convert columns containing 'num' or 'score' to numeric, replacing non-numeric symbols with NaN
for col in df.columns:
    if 'num' in col or 'score' in col:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace null values in numeric columns with the median
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Save the updated DataFrame to a new CSV file
df.to_csv('/Users/peterscheinsohn/Downloads/cleaned_sat-results.csv', index=False)

# Display the DataFrame to verify the changes
print(df.head())

      dbn                                    school name  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

   num of sat test takers  sat critical reading avg. score  \
0                    29.0                            355.0   
1                    91.0                            383.0   
2                    70.0                            377.0   
3                     7.0                            414.0   
4                    44.0                            390.0   

   sat math avg. score  sat writing avg. score  internal_school_id  \
0                404.0                   363.0              218160   
1                423.0                   366.0              268547   
2                402.0                   370.0          

/var/folders/h8/ldvp4k653rx3bjf1957sjxrm0000gn/T/ipykernel_1568/4142889638.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/h8/ldvp4k653rx3bjf1957sjxrm0000gn/T/ipykernel_1568/4142889638.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# Load your cleaned dataset
df = pd.read_csv('/Users/peterscheinsohn/Downloads/cleaned_sat-results_peter_s.csv')

# Database connection parameters
db_params = {
    'dbname': 'neondb',
    'user': 'neondb_owner',
    'password': 'npg_CeS9fJg2azZD',
    'host': 'ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech',
    'port': '5432',
    'sslmode': 'require'
}

# Create a SQLAlchemy engine with psycopg2 as the database driver
engine = create_engine(
    f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}",
    connect_args={'sslmode': db_params['sslmode']}
)

# Upload the DataFrame to the database using df.to_sql
df.to_sql('cleaned_sat-results_peter_s', engine, if_exists='replace', index=False)

print("Data uploaded successfully to the database.")

Data uploaded successfully to the database.
